In [17]:
import pandas as pd
data = pd.read_csv('thanksgiving-2015-poll-data.csv', encoding='Latin-1')
data.head(10)

,RespondentID,Do you celebrate Thanksgiving?,What is typically the main dish at your Thanksgiving dinner?,What is typically the main dish at your Thanksgiving dinner? - Other (please specify),How is the main dish typically cooked?,How is the main dish typically cooked? - Other (please specify),What kind of stuffing/dressing do you typically have?,What kind of stuffing/dressing do you typically have? - Other (please specify),What type of cranberry saucedo you typically have?,What type of cranberry saucedo you typically have? - Other (please specify),...,Have you ever tried to meet up with hometown friends on Thanksgiving night?,"Have you ever attended a ""Friendsgiving?""",Will you shop any Black Friday sales on Thanksgiving Day?,Do you work in retail?,Will you employer make you work on Black Friday?,How would you describe where you live?,Age,What is your gender?,How much total combined money did all members of your HOUSEHOLD earn last year?,US Region
0,4337954960,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,None,NaN,...,Yes,No,No,No,NaN,Suburban,18 - 29,Male,"$75,000 to $99,999",Middle Atlantic
1,4337951949,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Other (please specify),Homemade cranberry gelatin ring,...,No,No,Yes,No,NaN,Rural,18 - 29,Female,"$50,000 to $74,999",East South Central
2,4337935621,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Suburban,18 - 29,Male,"$0 to $9,999",Mountain
3,4337933040,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Homemade,NaN,...,Yes,No,No,No,NaN,Urban,30 - 44,Male,"$200,000 and up",Pacific
4,4337931983,Yes,Tofurkey,NaN,Baked,NaN,Bread-based,NaN,Canned,NaN,...,Yes,No,No,No,NaN,Urban,30 - 44,Male,"$100,000 to $124,999",Pacific
5,4337929779,Yes,Turkey,NaN,Roasted,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,No,NaN,Urban,18 - 29,Male,"$0 to $9,999",Pacific
6,4337924420,Yes,Turkey,NaN,Baked,NaN,Bread-based,NaN,Canned,NaN,...,No,No,Yes,Yes,No,Rural,18 - 29,Male,"$25,000 to $49,999",East North Central
7,4337916002,Yes,Turkey,NaN,Baked,NaN,Rice-based,NaN,Homemade,NaN,...,Yes,Yes,Yes,Yes,Yes,Rural,18 - 29,Male,Prefer not to answer,Mountain
8,4337914977,Yes,Turkey,NaN,Roasted,NaN,Bread-based,NaN,Canned,NaN,...,Yes,No,No,No,NaN,Urban,30 - 44,Male,"$75,000 to $99,999",Middle Atlantic
9,4337899817,Yes,Other (please specify),Turkey and Ham,Baked,NaN,Bread-based,NaN,Other (please specify),Both Canned and Homemade,...,No,No,No,Yes,Yes,Suburban,30 - 44,Male,"$25,000 to $49,999",East South Central


In [18]:
data['Do you celebrate Thanksgiving?'].value_counts()

Yes    980
No      78
Name: Do you celebrate Thanksgiving?, dtype: int64

In [19]:
# we're only interested in those who celebrate
data = data.loc[data['Do you celebrate Thanksgiving?'] == 'Yes']
data['Do you celebrate Thanksgiving?'].value_counts()

Yes    980
Name: Do you celebrate Thanksgiving?, dtype: int64

In [20]:
# list main dishes
data['What is typically the main dish at your Thanksgiving dinner?'].value_counts()

Turkey                    859
Other (please specify)     35
Ham/Pork                   29
Tofurkey                   20
Chicken                    12
Roast beef                 11
I don't know                5
Turducken                   3
Name: What is typically the main dish at your Thanksgiving dinner?, dtype: int64

In [21]:
#if you eat tofurkey, do you have gravy?
data.loc[data['What is typically the main dish at your Thanksgiving dinner?'] == 'Tofurkey']['Do you typically have gravy?']

4      Yes
33     Yes
69      No
72      No
77     Yes
145    Yes
175    Yes
218     No
243    Yes
275     No
393    Yes
399    Yes
571    Yes
594    Yes
628     No
774     No
820     No
837    Yes
860     No
953    Yes
Name: Do you typically have gravy?, dtype: object

In [22]:
# look at the counts
data.loc[data['What is typically the main dish at your Thanksgiving dinner?'] == 'Tofurkey']['Do you typically have gravy?'].value_counts()

Yes    12
No      8
Name: Do you typically have gravy?, dtype: int64

In [23]:
# find those who do not eat any pie
apple_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Apple'])
pumpkin_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pumpkin'])
pecan_isnull = pd.isnull(data['Which type of pie is typically served at your Thanksgiving dinner? Please select all that apply. - Pecan'])
no_pies = apple_isnull & pumpkin_isnull & pecan_isnull

no_pies.value_counts()

False    876
True     104
dtype: int64

In [24]:
# use this function to convert an age range to an integer value
# example use: string:'18-29' to an int: 18

def convert_single_string(string):
    if pd.isnull(string):
        return None
    age_str = string.split(' ')[0]
    age_str = int(age_str.replace('+',''))
    return age_str  

In [25]:
# create new column of integer ages using function
data['int_age'] = data['Age'].apply(convert_single_string)
data['int_age'].describe()

count    947.000000
mean      40.089757
std       15.352014
min       18.000000
25%       30.000000
50%       45.000000
75%       60.000000
max       60.000000
Name: int_age, dtype: float64

## Findings 
A good approximation of the mean age is 40.  We should be aware that this age data came from data where the particpant self selected their age from a range, for example, 18-29.  The lowest value of the range was used as the calculation, so it's safe to assume that the actual mean is higher.    

In [26]:
# use this function to convert the income bracket to an integer value
# example use:  convert string: '50,000 to 74,999' to int: 50000

def convert_string_to_income(string):
    if pd.isnull(string):
        return None
    # split on space
    income_str = string.split(' ')[0]
    
    # consider the "prefer not to answer case"
    if income_str == 'Prefer':
        return None
    
    # remove $ and , 
    income_str = income_str.replace('$','')
    income_str = int(income_str.replace(',',''))
    return income_str    

In [27]:
# create new column of integer incomes using function
data['int_income'] = data['How much total combined money did all members of your HOUSEHOLD earn last year?'].apply(convert_string_to_income)
data['int_income'].describe()

count       829.000000
mean      75965.018094
std       59068.636748
min           0.000000
25%       25000.000000
50%       75000.000000
75%      100000.000000
max      200000.000000
Name: int_income, dtype: float64

## Findings
Of the 829 survey particpants that responded to the question on household income, the average value is $75,965.  Like the age data, it is an approximation that is skewed downward.  The average value is higher than the national average, and the standard deviation is also high. 

In [28]:
# See how far people earning under $150,000 will travel
data.loc[data['int_income'] < 150000]['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         281
Thanksgiving is local--it will take place in the town I live in                     203
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    150
Thanksgiving is out of town and far away--I have to drive several hours or fly       55
Name: How far will you travel for Thanksgiving?, dtype: int64

In [29]:
# See how far people earning over $150,000 will travel
data.loc[data['int_income'] > 150000]['How far will you travel for Thanksgiving?'].value_counts()

Thanksgiving is happening at my home--I won't travel at all                         49
Thanksgiving is local--it will take place in the town I live in                     25
Thanksgiving is out of town but not too far--it's a drive of a few hours or less    16
Thanksgiving is out of town and far away--I have to drive several hours or fly      12
Name: How far will you travel for Thanksgiving?, dtype: int64

## Findings
A larger proportion of wealthy people stay at their home for thanksgiving.  

In [30]:
data.pivot_table(index = 'Have you ever tried to meet up with hometown friends on Thanksgiving night?', columns = 'Have you ever attended a "Friendsgiving?"', values = 'int_age')

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,42.283702,37.010526
Yes,41.475410,33.976744


In [31]:
data.pivot_table(index = 'Have you ever tried to meet up with hometown friends on Thanksgiving night?', columns = 'Have you ever attended a "Friendsgiving?"', values = 'int_income')

"Have you ever attended a ""Friendsgiving?""",No,Yes
Have you ever tried to meet up with hometown friends on Thanksgiving night?,,
No,78914.549654,72894.736842
Yes,78750.000000,66019.736842


## Findings
Those who attempt to incorporate friends into Thanksgiving tend to be younger and earn less money than those who do not. 

In [32]:
data.loc[data['What is typically the main dish at your Thanksgiving dinner?'] == 'Turkey']['How is the main dish typically cooked?'].value_counts()

Baked                     422
Roasted                   351
Fried                      41
Other (please specify)     34
I don't know               11
Name: How is the main dish typically cooked?, dtype: int64

In [33]:
data['What type of cranberry saucedo you typically have?'].value_counts()

Canned                    502
Homemade                  301
None                      146
Other (please specify)     25
Name: What type of cranberry saucedo you typically have?, dtype: int64

In [34]:
data['Will you employer make you work on Black Friday?'].value_counts()

Yes              43
No               20
Doesn't apply     7
Name: Will you employer make you work on Black Friday?, dtype: int64

In [37]:
data.pivot_table(columns = 'Will you employer make you work on Black Friday?', values = 'int_income')

Will you employer make you work on Black Friday?,Doesn't apply,No,Yes
int_income,57000.0,51944.444444,56666.666667


In [38]:
data.pivot_table(columns =  'Will you employer make you work on Black Friday?', values = 'int_age')

Will you employer make you work on Black Friday?,Doesn't apply,No,Yes
int_age,29.142857,30.75,32.714286
